In [ ]:
import numpy as np
from scipy.io import wavfile
import matplotlib.pyplot as plt
from scipy.stats import expon
%matplotlib notebook

In [ ]:
organrate, organ = wavfile.read('./resources/sample audio/organ.wav')
organ_l = organ[:, 0].astype(np.float64)
organ_r = organ[:, 1].astype(np.float64)

In [ ]:
N = organ_l.shape[0]
N

In [ ]:
def build_sinusoid_G(omegas, n):
    # number of frequencies
    J = omegas.shape[0]
    # 2 components for each freq, 1 row for each data point
    mat = np.zeros((n, 2*J))
    counter=0
    # build matrix of sinusiods at the normalised frequencies
    for i in range(J):
        freq = omegas[i]
        cosines = np.cos(np.arange(1, n+1)*freq)
        sines = np.sin(np.arange(1, n+1)*freq)
        mat[:, counter] = cosines
        counter += 1
        mat[:, counter] = sines
        counter += 1
    return mat

In [ ]:
omega = np.linspace(0.0001, 0.5, 1000)

In [ ]:
G = build_sinusoid_G(omega, N)

In [ ]:
def get_theta_ML(x, Gmat):
    # standard least squares expression
    return np.matmul(np.linalg.inv(np.matmul(Gmat.T, Gmat)), np.matmul(Gmat.T, x))

In [ ]:
theta_l = get_theta_ML(organ_l, G)
theta_r = get_theta_ML(organ_r, G)

In [ ]:
def get_theta_amp(thetas, omegas):
    thetas_amp = np.zeros(omegas.shape[0])
    # extract amplitude at each frequency as the root squared sum of a and b
    for i in range(omegas.shape[0]):
        thetas_amp[i] = np.square(thetas[2*i]) + np.square(thetas[2*i+1])
    return thetas_amp

In [ ]:
plt.figure()
plt.plot(omega, np.sqrt(get_theta_amp(theta_l, omega)), label='Left')
plt.plot(omega, np.sqrt(get_theta_amp(theta_r, omega)), label='Right')
plt.xlabel('Normalised frequency')
plt.ylabel('sqrt(a^2 + b^2)')
plt.legend()

In [ ]:
def get_posterior(x, Gmat, prior_mean, C_inv, noise_variance):
    # same prior expression
    phi = np.matmul(Gmat.T, Gmat) + noise_variance * C_inv
    Theta = np.matmul(Gmat.T, x) + noise_variance * np.matmul(C_inv, prior_mean)
    mean = np.matmul(np.linalg.inv(phi), Theta)
    covar = noise_variance * np.linalg.inv(phi)
    
    return mean, covar

In [ ]:
theta_map_l, _ = get_posterior(organ_l, G, np.zeros(2*omega.shape[0]), np.eye(2*omega.shape[0]), 1)
theta_map_r, _ = get_posterior(organ_r, G, np.zeros(2*omega.shape[0]), np.eye(2*omega.shape[0]), 1)

In [ ]:
plt.figure()
plt.plot(omega, get_theta_amp(theta_map_l, omega))
plt.plot(omega, get_theta_amp(theta_map_r, omega))

In [ ]:
def get_log_model_evidence(data, noise_variance, prior_mean, inv_prior_covar, Gen):
    Num_samps = data.shape[0]
    phi = np.matmul(Gen.T, Gen) + noise_variance * inv_prior_covar
    theta = np.matmul(Gen.T, data) + noise_variance * np.matmul(inv_prior_covar, prior_mean)
    th_map = np.matmul(np.linalg.inv(phi), theta)
    sign_C, log_det_C_inv = np.linalg.slogdet(inv_prior_covar)
    sign_P, log_det_phi = np.linalg.slogdet(phi)
    
    t1 = -Num_samps*np.log(2*np.pi)/2
    t2 = sign_C*log_det_C_inv/2
    t3 = -sign_P*log_det_phi/2
    t4 = -(Num_samps-2)*np.log(noise_variance)/2
    
    t5 = np.sum(np.square(data))
    t6 = noise_variance * np.linalg.multi_dot((prior_mean, inv_prior_covar, prior_mean))
    t7 = np.matmul(theta.T, th_map)
    
    
    return t1+t2+t3+t4 - (t5+t6-t7)/(2*noise_variance)

In [ ]:
model_ev = get_log_model_evidence(organ_l, 1, np.zeros(2*omega.shape[0]), np.eye(2*omega.shape[0]), G)
model_ev

In [ ]:
freq_prior = expon.pdf(omega, loc=0, scale=0.2)
plt.figure()
plt.plot(omega, freq_prior*np.exp(model_ev))